In [1]:
import pandas as pd
import os
import numpy as np
import json
from collections import Counter
from collections import OrderedDict
import ast
import sys
sys.path.append(os.path.abspath('../../'))
import dev_test.utils.cata_util as cata_util
from dev_test.catalog_verification.cad_functions import MARK_DRAW_FOO as mfuncs
from dev_test.catalog_verification.mark_catalogs import CANON_CATALOG as cc

In [2]:
# from catalog_verification/refined_catalogs read in a {tech}_refined.csv and return a dataframe
def read_refined_catalog(tech):
    path = os.path.abspath(f'../../dev_test/catalog_verification/refined_catalogs/{tech}_refined.csv')
    df = pd.read_csv(path)
    return df

In [3]:
# technologies = ['1222']

In [4]:
# df = read_refined_catalog(technologies[0])

In [5]:
# # loop through the rows of the dataframe and print catalog if not NaN
# with open("cata_origin_df.txt","w+") as f:
#     for index, row in df.iterrows():
#         if not pd.isnull(row['catalog']):
#             file, sheet, cell = df.loc[index, ['file', 'sheet', 'cell']]
#             df_origin = cata_util.get_df_info(technologies[0], file, sheet, cell)
#             # df_origin.to_csv(f, header=False, sep="\t", index=False)
#             np.savetxt(f, df_origin.values, fmt='%s', delimiter='\t')

In [6]:
# target_cata = 'CA2X9'
def get_df(df, technology, target_cata):
    lst_df = []
    for index, row in df.iterrows():
        if df.loc[index, 'catalog'] == target_cata:
            file, sheet, cell = df.loc[index, ['file', 'sheet', 'cell']]
            df_origin = cata_util.get_df_info(technology, file, sheet, cell)
            lst_df.append(df_origin)
    return lst_df


In [7]:
# lst_df = get_df(df, technologies[0], target_cata)

In [8]:
# mfunc_id = cata_util.get_catalog_seq(cc)[target_cata]

In [9]:
# mfunc_id

In [10]:
# get the function names from the column "SampleStartLayoutAssembler"
def get_func_arg_dict(df, mfunc_id):
    res = {}
    for m_i in mfunc_id:
        func_name = mfuncs[m_i]['name']
        row = df.loc[df['SampleStartLayoutAssembler'] == func_name]
        res[func_name] = {}
        for i, args in enumerate(mfuncs[m_i]['args']):
            # fill the ith argument of the function
            # print(row[:,i])
            res[func_name][args] = str(row.iloc[:,i+2].values[0])
    return res
    
    

In [11]:
# dict_test = []
# for i in range(len(lst_df)):
#     dict_test.append(get_func_arg_dict(lst_df[i], mfunc_id))

In [12]:
def _compare_dict(d1, d2, show_same=True):
    func_names = list(d1.keys())
    for func_name in func_names:
        for key in d1[func_name].keys():
            if d1[func_name][key] != d2[func_name][key]:
                print(f"diff: {key}, d1: {d1[func_name][key]}, d2: {d2[func_name][key]}")
            if show_same:
                print(f"same: {key}")

In [13]:
def args_equal(d1, d2):
    # except for 'Library.string' and 'CellName.string' in the 'StartLayoutAssembler' function
    # compare if two dictionaries are equal
    func_names = list(d1.keys())
    for func_name in func_names:
        for param in d1[func_name].keys():
            if (param not in ['CellName.string']) and (d1[func_name][param] != d2[func_name][param]):
                # print(f"diff: {param}, d1: {d1[func_name][param]}, d2: {d2[func_name][param]}")
                return False
    
    return True

In [14]:
# args_equal(dict_test[0], dict_test[1])

In [15]:
# for i in range(len(dict_test)):
#     for j in range(i+1, len(dict_test)):
#         if not args_equal(dict_test[i], dict_test[j]):
#             print(f"diff: {i}, {j}")

In [16]:
# _compare_dict(dict_test[0], dict_test[1])

In [17]:
def get_unique_args(dict_lst):
    # get the unique arguments from a list of dictionaries
    unique_args = [dict_lst[0]]
    for i in range(len(dict_lst)):
        for j in range(len(unique_args)):
            if args_equal(dict_lst[i], unique_args[j]):
                # if the args are the same, go check the next set of args
                break
        else:
            unique_args.append(dict_lst[i])
    return unique_args

In [18]:
# unique_args = get_unique_args(dict_test)

In [83]:
def main(technology):
    df = read_refined_catalog(technology)
    target_catas = [t for t in df['catalog'].unique() if str(t) != 'nan']
    freq = Counter(df['catalog'])

    for target_cata in target_catas[2:3]:
        print(f"target_cata: {target_cata}")
        print(f"freq: {freq[target_cata]}")
        lst_df = get_df(df, technology, target_cata)
        mfunc_id = cata_util.get_catalog_seq(cc)[target_cata]
        lst_dict = []  # stores the arguments of the functions
        for i in range(len(lst_df)):
            lst_dict.append(get_func_arg_dict(lst_df[i], mfunc_id))
        unique_args = get_unique_args(lst_dict)
    return unique_args
    

In [84]:
technologies = ['1222', '1227', '1276', '1278', '1280', '1282']
# target_catas = 'CA2X9'

In [86]:
unique_sets = main(technologies[3])


target_cata: CA2X9
freq: 24


In [87]:
len(unique_sets)

18